In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb
from collections import OrderedDict
import time
import copy
import re
import warnings
import sys
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

pd.set_option('display.max_columns', None)

In [ ]:
# my imports
sys.path.append('../src/')
from util.Utils import *
from GP.GP import *
from GP.FunctionTree import *
from GP.Objective import *

In [ ]:
''' load the data '''
# open
dbFile = '../data/baseballdata.db'
dbConn = sqlite3.connect(dbFile)
# query
sql = 'SELECT * FROM NLALRegularSeasonTeamStatsRanks;'
data = pd.read_sql(sql, dbConn)
display(data.head())
# close
dbConn.close()